In [1]:
import plotly.express as px
from pyspark.sql.functions import *

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('flights').getOrCreate()

23/01/02 13:49:24 WARN Utils: Your hostname, MacBook-Air-di-Teodoro.local resolves to a loopback address: 127.0.0.1; using 192.168.135.184 instead (on interface en0)
23/01/02 13:49:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/02 13:49:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:

df_cleaned = spark.read.csv('../../data.nosync/cleaned/cleaned_flights.csv/',inferSchema=True,header=True)

In [6]:
print(df_cleaned.count())
print(len(df_cleaned.columns))

6246739
43


In [3]:
df = spark.read.csv('../../data.nosync/', inferSchema=True, header=True)

In [15]:
import pandas as pd
features = pd.read_csv("../preprocessing/features.csv")["Features"].tolist()
features

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'FlightDate',
 'Reporting_Airline',
 'Tail_Number',
 'Flight_Number_Reporting_Airline',
 'Origin',
 'Dest',
 'CRSDepTime',
 'DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'DepDel15',
 'DepartureDelayGroups',
 'DepTimeBlk',
 'TaxiOut',
 'WheelsOff',
 'WheelsOn',
 'TaxiIn',
 'CRSArrTime',
 'ArrTime',
 'ArrDelay',
 'ArrDelayMinutes',
 'ArrDel15',
 'ArrivalDelayGroups',
 'ArrTimeBlk',
 'Cancelled',
 'Diverted',
 'CRSElapsedTime',
 'ActualElapsedTime',
 'AirTime',
 'Distance',
 'DistanceGroup']

In [16]:
df_cleaned = df.select(features)

In [17]:
# get the number of rows that contains in any column a null value
before = df_cleaned.count()
after = df_cleaned.na.drop().count()


In [19]:
total = before + after
null = before-after

In [22]:
# plot total and null as pie chart
fig = px.pie(names=["null", "total"], values=[null, total], title="Null values")
fig.show()

In [6]:
normal = df.filter(df['Diverted']==0).count()
diverted = df.filter(df['Diverted']==1).count()

In [9]:
# plot normal and diverted flights as pie chart with percentages
fig = px.pie(names=['Normal', 'Diverted'], values=[normal, diverted], title='Normal and diverted flights')
fig.show()

In [10]:
normal

6355322

In [11]:
diverted

14160